In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
def get_details(link):
    soup = BeautifulSoup(requests.get(link).text, 'lxml')
    try:
        details = soup.find('div', 'css-g5mtbi-Text').text.strip('\\n').strip('\\r')
        district = soup.find_all('a', class_ = 'css-tyi2d1')[-1].text.split()[-1]
        tags = [x.text for x in soup.find_all('p', class_='eu5v0x0')]
    except AttributeError:
        return ('delete',  '', '')
    return (process_post(details), district, tags)

def process_post(post):
    #Cleans post text. Joins into single string, removes /n
    post_str = "".join(post)
    return post_str.replace('\n', ' ').replace('\r', ' ')

In [3]:
links = pd.read_csv('Data/rent_links.csv')
links_filled=pd.read_parquet('Data/rent_details.parquet')

In [4]:
links_new=links[~links.id.isin(links_filled.id.values)]
links_new.loc[:, ['details','district', 'tags']]='', '', ''

C:\ProgramData\Miniconda3\envs\bs\lib\site-packages\pandas\core\indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


In [5]:
for index, row in tqdm(links_new.iterrows(), total=links_new.shape[0]):
    links_new.loc[index,['details', 'district', 'tags']] = get_details(row.url)

  0%|          | 0/2215 [00:00<?, ?it/s]

C:\ProgramData\Miniconda3\envs\bs\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


In [6]:
# update links on 'delete' id's
to_del=links_new[links_new.details == 'delete']
links[~links.id.isin(to_del.id.values)].to_csv('Data/rent_links.csv', index=False)
# remove delete from links_new. only then concat
links_new.drop(to_del.index, inplace=True)

C:\ProgramData\Miniconda3\envs\bs\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
links_upd = pd.concat([links_filled, links_new])
links_upd.to_parquet('Data/rent_details.parquet', index=False)